In [ ]:
"""横排EEQ数据-PCA"""

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

cats = ['AI','LC','LST','NDBSI','NDVI','WET']
cats_negtive = ['LST','NDBSI']
results = pd.DataFrame()
pc1 = pd.DataFrame()
percents = pd.DataFrame()

df = pd.read_excel('D:/ee-composite/src/sqlserver/data/E/Data_E_1.xlsx')
# print(df,df.shape)

df1 = df.query('catid=="AI"').sort_values('sn')
for yyyy in range(1985,2021):
    x = df1.loc[:,['sn','cityen','province','provincecn','city','class','classname','classid']]
    x['year'] = yyyy
    results = results.append(x)
results = results.reset_index()   
results.drop('index', axis=1, inplace=True) 
# print(results,results.shape)

for i in range(1,7,1):
    cc = cats[i-1]
    df1 = df.query('catid=="'+cc+'"').sort_values('sn')
    tmp = pd.DataFrame()
    for yyyy in range(1985,2021):
        tmp = tmp.append(df1.loc[:,[yyyy]].rename(columns={yyyy:cc}))
    tmp = tmp.reset_index()[cc]
    tmp = (tmp-tmp.mean())/tmp.std()
    if cc in cats_negtive :
        tmp = (tmp.max()-tmp)/(tmp.max()-tmp.min())
    else:
        tmp = (tmp-tmp.min())/(tmp.max()-tmp.min())

    results = pd.concat([results,tmp],axis=1)
    # print(results,results.shape)

pcadata = (results.loc[:,['sn','cityen','city','province','classname','classid','year','AI','LC','LST','NDBSI','NDVI','WET']]
            .rename(columns={'city':'citycn','cityen':'city'})
            .set_index(['sn','city','citycn','province','classname','classid','year']))
# print(pcadata,pcadata.shape)
pca = PCA(n_components=1)
tmp = pca.fit_transform(pcadata)
tmp = np.squeeze(tmp)
print(pca.explained_variance_ratio_)

pcadata = pcadata.reset_index()
pcadata['pc1'] = pd.Series(tmp)
pcadata['i'] = pcadata['year']-1984
# print(pcadata,pcadata.shape)

pc1 = pc1.append(pcadata)
pc1x = pc1['pc1']
pc1['pc1'] = (pc1x-pc1x.min())/(pc1x.max()-pc1x.min())
# print(pc1,pc1.shape)

pc1_city_mean = pc1.groupby(['sn','city','citycn','province','classname'])['pc1'].mean()
pc1_city_std = pc1.groupby(['sn','city','citycn','province','classname'])['pc1'].std()
pc1_year_mean = pc1.groupby('year').mean()
pc1_year_std = pc1.groupby('year').std()
# print(pc1_mean,pc1_mean.shape,pc1_std,pc1_std.shape)
# print(percents.mean(),percents.std())

pc1['iy'] = pc1['i']*pc1['pc1']
pc1['i2'] = pc1['i']*pc1['i']
pc1_slope = pc1.groupby(['sn','city','citycn','province','classname','classid']).sum()
pc1_slope['year'] = 2020
pc1_slope['slope'] = (36*pc1_slope['iy']-pc1_slope['i']*pc1_slope['pc1'])/(36*pc1_slope['i2']-pc1_slope['i']*pc1_slope['i'])
print(pc1_slope,pc1_slope.shape)

pc1.to_excel('Data_E_1_PC1.xlsx', sheet_name='PC1')
pc1_city_mean.to_excel('Data_E_1_PC1_city_mean.xlsx', sheet_name='PC1_mean')
pc1_city_std.to_excel('Data_E_1_PC1_city_mean_std.xlsx', sheet_name='PC1_mean_std')
pc1_year_mean.to_excel('Data_E_1_PC1_year_mean.xlsx', sheet_name='PC1_mean')
pc1_year_std.to_excel('Data_E_1_PC1_year_mean_std.xlsx', sheet_name='PC1_mean_std')
pc1_slope.to_excel('Data_E_1_PC1_slope.xlsx', sheet_name='slope')
# pc1.query('year==2020').groupby(['sn','city','citycn','province']).mean().to_excel('Data_E_1_PC1_citymean2020.xlsx', sheet_name='PC1_mean')
# pc1.groupby(['sn','city','citycn','province']).mean().to_excel('Data_E_1_PC1_citymean.xlsx', sheet_name='PC1_mean')
# pc1.groupby(['sn','city','citycn','province']).std().to_excel('Data_E_1_PC1_citymeanstd.xlsx', sheet_name='PC1_meanstd')